# XGBoost Hyperparams Optimisation using Optuna


It is based on the [EDA which makes sense ⭐️⭐️⭐️⭐️⭐️](https://www.kaggle.com/code/ambrosm/amex-eda-which-makes-sense).
Based On - [LightGBM Quickstart](https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats
import warnings
from colorama import Fore, Back, Style
import gc

from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibrationDisplay
from lightgbm import LGBMClassifier, log_evaluation

plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
plt.rcParams['text.color'] = 'w'

INFERENCE = True # set to False if you only want to cross-validate


In [ ]:
def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return 'amex', amex_metric(y_true, y_pred)

# Reading and preprocessing the data

We read the data from @munumbutt's [AMEX-Feather-Dataset](https://www.kaggle.com/datasets/munumbutt/amexfeather). Then we create two groups of features:
- Selected features averaged over all statements of a customer
- Selected features taken from the last statement of a customer

The code has been optimized for memory efficiency rather than readability. In particular, `.iloc[mask_array, columns]` needs much less RAM than the groupby construction used in the previous version of the notebook.

Preprocessing for LightGBM is much simpler than for neural networks:
1. Neural networks can't process missing values; LightGBM handles them automatically.
1. Categorical features need to be one-hot encoded for neural networks; LightGBM handles them automatically.
1. With neural networks, you need to think about outliers; tree-based algorithms deal with outliers easily.
1. Neural networks need scaled inputs; tree-based algorithms ignore the scaling.

In [ ]:
%%time
features_avg = ['B_1', 'B_11', 'B_16', 'B_17', 'B_18', 'B_2', 'B_20',
                'B_28', 'B_3', 'B_4', 'B_5', 'B_7', 'B_9', 'D_112',
                'D_121', 'D_141', 'D_39', 'D_41', 'D_42', 'D_43',
                'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 
                'D_50', 'D_51', 'D_53', 'D_54', 'D_56', 'D_58', 
                'D_59', 'D_60', 'D_91', 'P_2', 'P_3', 'R_1', 'R_2', 
                'R_27', 'R_3', 'R_7', 'S_11', 'S_26', 'S_3', 'S_5']
features_last = ['B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_15', 'B_16',
                 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_22', 'B_23',
                 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3',
                 'B_32', 'B_33', 'B_36', 'B_38', 'B_39', 'B_4', 'B_40',
                 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9',
                 'D_102', 'D_103', 'D_105', 'D_106', 'D_107', 'D_109',
                 'D_112', 'D_115', 'D_117', 'D_118', 'D_119', 'D_120',
                 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 
                 'D_129', 'D_132', 'D_133', 'D_135', 'D_136', 'D_137', 
                 'D_140', 'D_141', 'D_143', 'D_145', 'D_39', 'D_41',
                 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48',
                 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55',
                 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63',
                 'D_64', 'D_66', 'D_70', 'D_72', 'D_73', 'D_74', 'D_75',
                 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_82', 'D_83',
                 'D_84', 'D_86', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96',
                 'P_2', 'P_3', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13',
                 'R_14', 'R_15', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 
                 'R_21', 'R_22', 'R_24', 'R_25', 'R_26', 'R_27', 'R_3',
                 'R_4', 'R_5', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12',
                 'S_13', 'S_15', 'S_17', 'S_20', 'S_22', 'S_23', 
                 'S_24', 'S_25', 'S_26', 'S_27', 'S_3', 'S_5', 'S_6',
                 'S_7', 'S_8', 'S_9']

train_test = [None, None] # first element is train, second element is test
for i in [1, 0] if INFERENCE else [0]:
    train_test[i] = pd.read_feather(['../input/amexfeather/train_data.ftr',
                                     '../input/amexfeather/test_data.ftr'][i])
    cid = pd.Categorical(train_test[i].pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer
    if i == 0: # train
        target = train_test[0].loc[last, 'target']
    gc.collect()
    print('Read', i)
    df_avg = (train_test[i][features_avg]
              .groupby(cid)
              .mean()
              .rename(columns={f: f"{f}_avg" for f in features_avg})
             )
    gc.collect()
    print('Computed avg', i)
    train_test[i] = (train_test[i].loc[last, features_last]
                     .rename(columns={f: f"{f}_last" for f in features_last})
                     .set_index(np.asarray(cid[last]))
                    )
    gc.collect()
    print('Computed last', i)
    train_test[i] = pd.concat([train_test[i], df_avg], axis=1)
    del df_avg, cid, last

train, test = tuple(train_test)
del train_test
if INFERENCE: print('Shapes:', train.shape, target.shape, test.shape)


# Cross-validation

We cross-validate with a five-fold StratifiedKFold.

Notice that lightgbm logs the validation score with the competition's scoring function every ten iterations.

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
#train['B_38_last']

In [ ]:
train = train.drop(['B_38_last', 'D_117_last', 'D_120_last', 'D_63_last', 'D_64_last', 'D_66_last'],axis=1)

In [ ]:
import optuna

In [ ]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.2, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-2, 10.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 18, 100.0)
    subsample = trial.suggest_float("subsample", 0.55, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 0.45)
    gamma = trial.suggest_float("gamma", 0.7,1.8)
    max_depth = trial.suggest_int("max_depth", 1, 7)
    min_child_weight = trial.suggest_int("min_child_weight", 3, 9)
    n_estimators = trial.suggest_int("n_estimators", 1000, 8000,step=200)
    features = [f for f in train.columns if f != 'customer_ID' and f != 'target']
    ONLY_FIRST_FOLD = False
    
    
    
    score_list = []
    y_pred_list = []
    kf = StratifiedKFold(n_splits=5)
    for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
        start_time = datetime.datetime.now()
        
        X_tr = train.iloc[idx_tr][features]
        X_va = train.iloc[idx_va][features]
        y_tr = target.iloc[idx_tr]
        y_va = target.iloc[idx_va]
        
        
        model = XGBClassifier(random_state=1,
        tree_method="gpu_hist",
        gpu_id=0,
        predictor="gpu_predictor",
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
        gamma=gamma,
                                  min_child_weight = min_child_weight,
                                  booster = 'gbtree'
    )

        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            model.fit(X_tr, y_tr,
                      eval_set = [(X_tr, y_tr),(X_va, y_va)], verbose=20000
                      )

        y_va_pred = model.predict_proba(X_va)[:,1]
        score = amex_metric(y_va.values, y_va_pred)
        n_trees = model.get_booster().best_iteration
        #if n_trees is None: n_trees = model.n_estimators
        '''print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | {str(datetime.datetime.now() - start_time)[-12:-7]} |"
              f" {n_trees:5} trees |"
              f"                Score = {score:.5f}{Style.RESET_ALL}")'''
        score_list.append(score)
        

        if ONLY_FIRST_FOLD: break # we only want the first fold

    #print(f"{Fore.GREEN}{Style.BRIGHT}OOF Score:                       {np.mean(score_list):.5f}{Style.RESET_ALL}")
    return np.mean(score_list)

In [ ]:
# %%time
# # Cross-validation of the classifier





# def my_booster(random_state=1, n_estimators=195):
#     return LGBMClassifier(n_estimators=n_estimators,
#                           min_child_samples=2400,
#                           num_leaves=127,
#                           max_bins=511, random_state=random_state)

# def xgb_boost():
    
    

#     xgb_params = { 'random_state': 1, 
#     'n_jobs': 4,
#     'booster': 'gbtree',
#     'n_estimators': 4000,
#     'learning_rate': 0.034682894846408095,
#     'reg_lambda': 1.224383455634919,
#     'reg_alpha': 36.043214512614476,
#     'subsample': 0.9219010649982458,
#     'colsample_bytree': 0.11247495917687526,
#     'max_depth': 3,
#     'min_child_weight': 6,
#               'tree_method': 'gpu_hist',
#                   'feval': lgb_amex_metric
# }
                         


#     return XGBClassifier(**xgb_params)




# #print(f"{len(features)} features")
# score_list = []
# y_pred_list = []
# kf = StratifiedKFold(n_splits=5)
# for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
#     start_time = datetime.datetime.now()
#     model = xgb_boost()
#     X_tr = train.iloc[idx_tr][features]
#     X_va = train.iloc[idx_va][features]
#     y_tr = target.iloc[idx_tr]
#     y_va = target.iloc[idx_va]
    
    
#     with warnings.catch_warnings():
#         warnings.filterwarnings('ignore', category=UserWarning)
#         model.fit(X_tr, y_tr,
#                   eval_set = [(X_tr, y_tr),(X_va, y_va)], verbose=2000
#                   )
                  
#     y_va_pred = model.predict_proba(X_va)[:,1]
#     score = amex_metric(y_va.values, y_va_pred)
#     n_trees = model.get_booster().best_iteration
#     if n_trees is None: n_trees = model.n_estimators
#     print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | {str(datetime.datetime.now() - start_time)[-12:-7]} |"
#           f" {n_trees:5} trees |"
#           f"                Score = {score:.5f}{Style.RESET_ALL}")
#     score_list.append(score)
#     n_estimators += 75
#     if INFERENCE:
#         y_pred_list.append(model.predict_proba(test[features])[:,1])
        
#     if ONLY_FIRST_FOLD: break # we only want the first fold
    
# print(f"{Fore.GREEN}{Style.BRIGHT}OOF Score:                       {np.mean(score_list):.5f}{Style.RESET_ALL}")

# Prediction histogram

In [ ]:
%%time
study = optuna.create_study(direction="maximize",study_name = 'xgbclf1_')
study.optimize(objective, n_trials=2)

# Calibration diagram

The calibration diagram shows how the model predicts the default probability of customers:

In [ ]:
study.best_params

# Submission

We submit the mean of the five predictions.